In [1]:
from sklearn.datasets import *
import gc
from LightGBM.LightGBM_ToolBox_ver_2 import *

%load_ext autoreload
%autoreload 2

### Load data:

In [2]:
t = load_boston()

data = pd.DataFrame(load_boston().data)
data.columns = t.feature_names
target = load_boston().target

In [3]:
print(data.shape)
data.head()

(506, 13)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [4]:
print('Randomly change some data to np.nan...')
random_idx = np.random.choice(data.index, size=int(data.shape[0]*0.2))
print(len(random_idx))

Randomly change some data to np.nan...
101


In [5]:
print('Change partial data to NULL...')
data.loc[random_idx, 'AGE'] = np.nan
gc.collect()

Change partial data to NULL...


0

In [6]:
data['AGE'].isnull().sum()

90

### Test modules:

In [7]:
mylight = MyLight(objective='regression')

In [8]:
mylight.get_log()

[02/03/2019 01:11:46 PM (Local Time)] INFO : Start logging...
[02/03/2019 01:11:46 PM (Local Time)] INFO : Update instance booster params to: {'subsample_for_bin': 200000, 'class_weight': None, 'min_split_gain': 0.0, 'subsample': 1.0, 'num_leaves': 31, 'reg_lambda': 0.0, 'n_estimators': 100, 'boosting_type': 'gbdt', 'min_child_samples': 20, 'colsample_bytree': 1.0, 'subsample_freq': 0, 'max_depth': -1, 'reg_alpha': 0.0, 'min_child_weight': 0.001, 'objective': 'regression'}



In [9]:
print(mylight.lgb_default_metrics)
print(mylight.lgb_default_objectives)

['l1', 'mean_absolute_error', 'mae', 'regression_l1', 'l2', 'mean_squared_error', 'mse', 'regression_l2', 'regression', 'l2_root', 'root_mean_squared_error', 'rmse', 'quantile', 'mape', 'mean_absolute_percentage_error', 'huber', 'fair', 'poisson', 'gamma', 'gamma_deviance', 'tweedie', 'ndcg', 'map', 'auc', 'binary_logloss', 'binary_error', 'multi_logloss', 'multiclass', 'softmax', 'multiclassova', 'multiclass_ova', 'ova', 'ovr', 'multi_error', 'xentropy', 'cross_entropy', 'xentlambda', 'cross_entropy_lambda', 'kldiv', 'kullback_leibler']
['regression', 'regression_l1', 'huber', 'fair', 'poisson', 'quantile', 'mape', 'gamma', 'tweedie', 'binary', 'multiclass', 'multiclassova', 'cross_entropy', 'cross_entropy_lambda', 'lambdarank']


In [10]:
n_rounds = 500
learning_rates=[0.1, 0.001, 0.0001]
for i in range(n_rounds-3):
    learning_rates.append(0.00001)

In [11]:
model, feats_imp = mylight.fit(data, target, learning_rates=learning_rates, verbose_eval=100, eval_metric=['rmse'], num_iterations=n_rounds, n_jobs=6,\
                   analyze_feature_importance=True)

model training
Training until validation scores don't improve for 100 rounds.
[100]	Training's rmse: 8.63662	Validation's rmse: 7.92799
[200]	Training's rmse: 8.62964	Validation's rmse: 7.92237
[300]	Training's rmse: 8.62266	Validation's rmse: 7.91675


C:\Users\zoomz\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[400]	Training's rmse: 8.6157	Validation's rmse: 7.91115
[500]	Training's rmse: 8.60875	Validation's rmse: 7.90558
Did not meet early stopping. Best iteration is:
[500]	Training's rmse: 8.60875	Validation's rmse: 7.90558
*********************************************
For model training, Time cost: 0.39 second(s)
*********************************************


In [12]:
trained_models, preds = mylight.kfolds_cv_ensemble(data, target, learning_rate=0.0001, num_iterations=100, save_models=True)

Start K-Fold cross validation (ensemble)...

For fold 1...

model training
Training until validation scores don't improve for 100 rounds.
[100]	Training's rmse: 9.32954	Validation's rmse: 8.67494
Did not meet early stopping. Best iteration is:
[100]	Training's rmse: 9.32954	Validation's rmse: 8.67494
*********************************************
For model training, Time cost: 0.10 second(s)
*********************************************

For fold 2...

model training
Training until validation scores don't improve for 100 rounds.
[100]	Training's rmse: 9.03484	Validation's rmse: 9.28079
Did not meet early stopping. Best iteration is:
[100]	Training's rmse: 9.03484	Validation's rmse: 9.28079
*********************************************
For model training, Time cost: 0.09 second(s)
*********************************************

For fold 3...

model training
Training until validation scores don't improve for 100 rounds.
[100]	Training's rmse: 8.97289	Validation's rmse: 9.39678
Did not meet

In [13]:
mylight.bayes_opt_X = data
mylight.bayes_opt_y = target
mylight.bayes_opt_metrics = 'rmse'

In [14]:
res = mylight.bayes_tuning()

Failed in cross validation by using user-provided data set. Error: local variable '_feature_name' referenced before assignment
Failed in Bayesian optimization. Error: local variable '_feature_name' referenced before assignment


UnboundLocalError: local variable '_feature_name' referenced before assignment